In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from data import dataloader
import json
from neuralnet import nn
import pixiedust_node

In [ ]:
pilot_data = dataloader.break_out_blocks(
    dataloader.load_from_firebase()
)

---

In [ ]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))

---

In [ ]:
with open(
    'data/targets.json',
    'r'
) as fp:
    targets = json.load(
        fp
    )[0]

In [ ]:
data = {}
for target in targets:
    data[target] = {
        "train": nn.define_trainer_data(
            pilot_data,
            {
                "target": [target],
                "offtarget": targets[target]
            },
            [
                "thermopile1",
                "thermopile2",
                "thermopile3",
                "thermopile4"
            ],
            list(range(1, 7)),
            100
        ),
        "test_true": nn.define_activation(
            pilot_data,
            [target],
            [
                "thermopile1",
                "thermopile2",
                "thermopile3",
                "thermopile4"
            ],
            list(range(1, 7)),
            exclude=100
        ),
        "test_false": nn.define_activation(
            pilot_data,
            targets[target],
            [
                "thermopile1",
                "thermopile2",
                "thermopile3",
                "thermopile4"
            ],
            list(range(1, 7)),
            exclude=100
        )
    }

---

In [ ]:
%%node
var synaptic = require('synaptic');
var Neuron = synaptic.Neuron,
	Layer = synaptic.Layer,
	Network = synaptic.Network,
	Trainer = synaptic.Trainer,
	Architect = synaptic.Architect;

In [ ]:
%%node
/**
 * @function Perceptron - Function to create synaptic perceptron
 * @see https://github.com/cazala/synaptic/blob/master/README.md
 * @param input
 * @param hidden
 * @param output
 * @this
 */
function Perceptron(input, hidden, output){
	// create the layers
	var inputLayer = new Layer(input);
	var hiddenLayer = new Layer(hidden);
	var outputLayer = new Layer(output);

	// connect the layers
	inputLayer.project(hiddenLayer);
	hiddenLayer.project(outputLayer);

	// set the layers
	this.set({
		input: inputLayer,
		hidden: [hiddenLayer],
		output: outputLayer
	});
}

// extend the prototype chain
Perceptron.prototype = new Network();
Perceptron.prototype.constructor = Perceptron;

function runPerceptronOnList(perceptron, listOfInputs){
    var outputs = [];
    for(var i=0; i<listOfInputs.length; i++){
        outputs.push(perceptron.activate(listOfInputs[i]));
    }
    return(outputs);
}

In [ ]:
%%node
var perceptron5 = new Perceptron(4,5,1);
var perceptron7 = new Perceptron(4,7,1);
var trainer5 = new Trainer(perceptron5);
var trainer7 = new Trainer(perceptron7);

trainer5.train(cheek_train);
trainer7.train(cheek_train);

In [ ]:
%%node
print(runPerceptronOnList(perceptron5, cheek_test_true))